# Câu 2: Classification – Coronavirus tweets NLP (1.0 điểm)
Use data in **covid_text_classification** folder to build a model to determine if an 
OriginalTweet is Extremely Negative/ Negative/ Neutral/ Positive/ Extremely Positive.
<br>Read more information here:
<br>https://www.kaggle.com/datatattle/covid-19-nlp-text-classification

In [ ]:
#@title Install Pakages
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark  # Tìm Spark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,365 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packa

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/covid_text_classification'

/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Du lieu cung cap/covid_text_classification


In [ ]:
#@title Import pakages
import pyspark
import pandas as pd
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import isnan, count, regexp_replace, regexp_extract, col, datediff, when, length, log, array, lit, trim
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from datetime import datetime

from pyspark.ml.feature import Binarizer,Bucketizer, OneHotEncoder, StringIndexer, Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel, GBTClassifier
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.classification import LinearSVC, LinearSVCModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [ ]:
sc = SparkContext()
spark = SparkSession.builder.appName('cau2').getOrCreate()

In [ ]:
data = spark.read.csv('Corona_NLP_train.csv', inferSchema=True, header=True)
data.show(5, False)

+--------+------------+-------------------------------------------------------------------------------+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|UserName|ScreenName  |Location                                                                       |TweetAt   |OriginalTweet                                                                                                                                                                                                                                |Sentiment|
+--------+------------+-------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Create a temporary view of data
# data.createOrReplaceTempView('df')
df = data.select('Location','TweetAt','OriginalTweet','Sentiment')

In [ ]:
# Các giá trị trên cột OriginalTweet không thể là giá trị null
# => Lọc các giá trị khác null trên cột OriginalTweet
# query = 'SELECT * FROM df WHERE OriginalTweet != ""'
# df = spark.sql(query)
# df.show(5)

In [ ]:
df.count()

68046

In [ ]:
df.printSchema()

root
 |-- Location: string (nullable = true)
 |-- TweetAt: string (nullable = true)
 |-- OriginalTweet: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [ ]:
#Kiểm tra dữ liệu NaN, null
df.select([count(when(isnan(c),c)).alias(c) for c in df.columns]).toPandas().T

,0
Location,0
TweetAt,0
OriginalTweet,0
Sentiment,0


In [ ]:
# => Không có giá trị nan

In [ ]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T

,0
Location,33799
TweetAt,26311
OriginalTweet,26663
Sentiment,39429


In [ ]:
# Các giá trị trên OriginalTweet không thể là giá trị null => Cần xóa các giá trị null
df = df.na.drop(subset=["OriginalTweet"])
df.count()

41383

In [ ]:
# => Các giá trị null trên Sentiment sẽ được lọc ra sau khi assembler để làm dữ liệu kiểm tra sau khi xây dựng mô hình

In [ ]:
#Kiểm tra dữ liệu trùng. Xóa dữ liệu trùng
num_rows = df.count()
num_dist_rows = df.distinct().count()
dup_rows = num_rows - num_dist_rows

In [ ]:
display(num_rows, num_dist_rows, dup_rows)

41383

41331

52

In [ ]:
# Xóa dữ liệu duplicate
df = df.drop_duplicates()
df.count()

41331

In [ ]:
# Kiểm tra lại số lượng giá trị null
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T

,0
Location,8554
TweetAt,0
OriginalTweet,0
Sentiment,12715


In [ ]:
# Đổi KDL trên cột TweetAt về datetime
func =  udf(lambda x: datetime.strptime(x, '%d-%m-%Y'), types.DateType())
df = df.withColumn('TweetAt_Date', func(col('TweetAt')))
df.show(3)


+--------------+----------+--------------------+------------------+------------+
|      Location|   TweetAt|       OriginalTweet|         Sentiment|TweetAt_Date|
+--------------+----------+--------------------+------------------+------------+
|Makati, Manila|16-03-2020|All month there h...|           Neutral|  2020-03-16|
|      Reno, NV|16-03-2020|Of all the things...|Extremely Negative|  2020-03-16|
|      Reno, NV|16-03-2020|Yall think #coron...|          Negative|  2020-03-16|
+--------------+----------+--------------------+------------------+------------+
only showing top 3 rows



In [ ]:
df.groupBy('Location').agg({'Sentiment':'count','TweetAt':'count'}).show(truncate=False)

+------------------------------+----------------+--------------+
|Location                      |count(Sentiment)|count(TweetAt)|
+------------------------------+----------------+--------------+
|UR MOMZ                      |1               |1             |
|Bangalore                     |19              |28            |
|Northumberland                |1               |1             |
|Mumbai, Maharashtra           |3               |6             |
|Luton, England                |1               |3             |
|St Petersburg, FL             |9               |13            |
|Ferrara, Emilia Romagna       |1               |1             |
|Pleasanton, CA                |2               |2             |
|Brisbane, Australia           |4               |9             |
|Downtown Flushing, Queens     |1               |1             |
|Heaven                        |1               |2             |
|Chesterfield, VA              |4               |4             |
|Lowell Ma.              

In [ ]:
# Tiền xử lý chuỗi trên OriginalTweet
df = df.withColumn("OriginalTweet_clean", regexp_replace(col("OriginalTweet"), "[^a-zA-Z]", " "))
df.select('OriginalTweet','OriginalTweet_clean','Sentiment').show(5)

+--------------------+--------------------+------------------+
|       OriginalTweet| OriginalTweet_clean|         Sentiment|
+--------------------+--------------------+------------------+
|All month there h...|All month there h...|           Neutral|
|Of all the things...|Of all the things...|Extremely Negative|
|Yall think #coron...|Yall think  coron...|          Negative|
|My prayers ?? go ...|My prayers    go ...|              null|
|@HaileeSteinfeld ...| HaileeSteinfeld ...|Extremely Positive|
+--------------------+--------------------+------------------+
only showing top 5 rows



In [ ]:
# Chọn các thuộc tính phù hợp: input - OriginalTweet_clean, output - Sentiment
df_pre = df.select('OriginalTweet_clean','Sentiment')
df_pre.groupBy('Sentiment').count().show()

+--------------------+-----+
|           Sentiment|count|
+--------------------+-----+
|    online education|    1|
| consumer and mor...|    1|
|              #virus|    1|
| only a few preli...|    1|
| potatoes &amp; v...|    1|
| Vaccines and Tre...|    1|
|      Mumbai or Pune|    1|
|"" as shoppers ac...|    1|
| spot the scams""...|    1|
|000 tests for COV...|    1|
| claiming the new...|    1|
| as lower oil pri...|    1|
| they should add ...|    1|
| we are confronte...|    1|
|  closed the borders|    1|
| IFB vice preside...|    1|
| Big Tech could b...|    1|
| Bakery etc. This...|    1|
| but we have to a...|    1|
|                  so|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# => Các giá trị trên Sentiment tồn tại các giá trị không xác định trong các class Extremely Negative/ Negative/ Neutral/ Positive/ Extremely Positive
# Cần xử lý các giá trị không xác định, chuyển về giá trị null

In [ ]:
import numpy as np
df_pre = df_pre.withColumn('label', when(df.Sentiment=='Extremely Negative', 0)\
                           .when(df.Sentiment=='Negative', 1)\
                           .when(df.Sentiment=='Neutral', 2)\
                           .when(df.Sentiment=='Positive', 3)\
                           .when(df.Sentiment=='Extremely Positive', 4)\
                           .otherwise(np.nan))

In [ ]:
df_pre.show()

+--------------------+------------------+-----+
| OriginalTweet_clean|         Sentiment|label|
+--------------------+------------------+-----+
|All month there h...|           Neutral|  2.0|
|Of all the things...|Extremely Negative|  0.0|
|Yall think  coron...|          Negative|  1.0|
|My prayers    go ...|              null|  NaN|
| HaileeSteinfeld ...|Extremely Positive|  4.0|
|No shortage of fo...|Extremely Negative|  0.0|
|In resonse to COV...|          Positive|  3.0|
|COVID    is a Wak...|          Negative|  1.0|
|Amidst all the  r...|              null|  NaN|
| Amazon Is Hiring...|Extremely Positive|  4.0|
| SkyIreland hiya ...|Extremely Positive|  4.0|
|One of the only  ...|           Neutral|  2.0|
|California govern...|          Positive|  3.0|
|We ve rightly bee...|          Positive|  3.0|
|Good stuff Mongol...|              null|  NaN|
|Scary Gun sales g...|Extremely Negative|  0.0|
|Just wondering if...|           Neutral|  2.0|
|The Push  amp  Pu...|          Negative

In [ ]:
df_pre.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 3751|
|  NaN|13369|
|  1.0| 6857|
|  4.0| 4412|
|  3.0| 7718|
|  2.0| 5224|
+-----+-----+



## Tạo Vector Assembler

In [ ]:
# Chuẩn hóa dữ liệu 'OriginalTweet_clean'
tokenizer = Tokenizer(inputCol='OriginalTweet_clean', outputCol='token_text')
stopremove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='cv')
idf = IDF(inputCol='cv', outputCol='tf-idf')


In [ ]:

assembler = VectorAssembler(inputCols=['tf-idf'],
                           outputCol='features')

## Chuẩn bị dữ liệu train, test

In [ ]:
# Lọc dữ liệu null để làm dl predict
new_data = df_pre.select("OriginalTweet_clean","label").filter(df_pre.label == np.nan)

new_data.show(3)

+--------------------+-----+
| OriginalTweet_clean|label|
+--------------------+-----+
|My prayers    go ...|  NaN|
|Amidst all the  r...|  NaN|
|Good stuff Mongol...|  NaN|
+--------------------+-----+
only showing top 3 rows



In [ ]:
# dữ liệu không null được dùng để train
data_final = df_pre.na.drop()
data_final.count()

27962

In [ ]:
data_final = data_final.select("OriginalTweet_clean", 'label')
data_final.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 3751|
|  1.0| 6857|
|  4.0| 4412|
|  3.0| 7718|
|  2.0| 5224|
+-----+-----+



* Cân bằng dữ liệu trên các lớp: Extremely Negative (0)/ Negative(1)/ Neutral(2)/ Positive(3)/ Extremely Positive(4) của tập train

In [ ]:
Positive = data_final.filter(col("label") == 3.0) 
extremely_negative = data_final.filter(col("label") == 0.0)
negative = data_final.filter(col("label") == 1.0)
neural = data_final.filter(col("label") == 2.0)
extremely_positive = data_final.filter(col("label") == 4.0)

ratio_1 = int(Positive.count()/extremely_negative.count()) 
ratio_2 = int(Positive.count()/negative.count()) 
ratio_3 = int(Positive.count()/neural.count()) 
ratio_4 = int(Positive.count()/extremely_positive.count()) 

print("Positive/extremely_negative/negative/neural/extremely_positive: ", ratio_1, ratio_2, ratio_3, ratio_4) 

Positive/extremely_negative/negative/neural/extremely_positive:  2 1 1 1


In [ ]:
# Resample extremely_negative
a1 = range(ratio_1)

oversampled_extremely_negative = extremely_negative.withColumn('dummy',
                                              explode(array([lit(x) for x in a1]))).drop('dummy')
data_final_resample = Positive.unionAll(oversampled_extremely_negative)

a2 = range(ratio_2)
oversampled_negative = negative.withColumn('dummy',
                                              explode(array([lit(x) for x in a2]))).drop('dummy')
data_final_resample = data_final_resample.unionAll(oversampled_negative)

a3 = range(ratio_3)
oversampled_neural = neural.withColumn('dummy',
                                              explode(array([lit(x) for x in a3]))).drop('dummy')
data_final_resample = data_final_resample.unionAll(oversampled_neural)

a4 = range(ratio_4)
oversampled_extremely_positive = extremely_positive.withColumn('dummy',
                                              explode(array([lit(x) for x in a4]))).drop('dummy')
data_final_resample = data_final_resample.unionAll(oversampled_extremely_positive)

In [ ]:
data_final_resample.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 7502|
|  1.0| 6857|
|  4.0| 4412|
|  3.0| 7718|
|  2.0| 5224|
+-----+-----+



## Tạo tập dữ liệu Train và Test

In [ ]:
training, testing = data_final_resample.randomSplit([0.8,0.2])

In [ ]:
training.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 6018|
|  1.0| 5462|
|  4.0| 3544|
|  3.0| 6150|
|  2.0| 4103|
+-----+-----+



In [ ]:
testing.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 1484|
|  1.0| 1395|
|  4.0|  868|
|  3.0| 1568|
|  2.0| 1121|
+-----+-----+



## Áp dụng các mô hình Classification
+ Logistic Regression
+ Naive Bayes

In [ ]:
# init classification model
lg = LogisticRegression(featuresCol = "features",
                      labelCol = "label", 
                      maxIter = 20, 
                      regParam=0.3, 
                      predictionCol = "prediction")
nb = NaiveBayes(featuresCol = "features",
                smoothing=1.0,
                modelType="multinomial",
                labelCol = "label",
                predictionCol = "prediction")


In [ ]:
# init pipeline for models
# logisticRegression pipeline
pipeline_lg = Pipeline(stages=[tokenizer,
                            stopremove,
                            count_vec,
                            idf,
                            assembler, lg])

# Naive Bayes pipeline
pipeline_nb = Pipeline(stages=[tokenizer,
                            stopremove,
                            count_vec,
                            idf,
                            assembler, nb])


In [ ]:
# fit pipeline
lg_model = pipeline_lg.fit(training)
nb_model = pipeline_nb.fit(training)

In [ ]:
#apply transform
test_result_lg = lg_model.transform(testing)
test_result_nb = nb_model.transform(testing)

In [ ]:
# avaluate models
acc_eval = MulticlassClassificationEvaluator(labelCol="label",
                                        predictionCol="prediction",
                                        metricName="accuracy")
acc_lg = acc_eval.evaluate(test_result_lg)
print("Accuracy of Logistic model at predicting sentiment: {0:2.2f}%".format(acc_lg*100))
print('-'*80)
acc_nb = acc_eval.evaluate(test_result_nb)
print("Accuracy of Naive Bayes model at predicting sentiment: {0:2.2f}%".format(acc_nb*100))
print('-'*80)


Accuracy of Logistic model at predicting sentiment: 53.48%
--------------------------------------------------------------------------------
Accuracy of Naive Bayes model at predicting sentiment: 49.84%
--------------------------------------------------------------------------------


**Nhận xét:**
+ Model có độ chính xác chưa tốt
+ LogisticRegression có độ chính xác là 53.48% , Naive Bayes có độ chính xác là 49.84%
+ Lựa chọn LogisticRegression model để dự đoán trên các tập dữ liệu test

## Đánh giá model trên test set

In [ ]:
# Create a confusion matrix
confusion_matrix = test_result_lg.groupBy('label', 'prediction').count()

In [ ]:
confusion_matrix.filter(confusion_matrix['label']==confusion_matrix['prediction']).show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  500|
|  2.0|       2.0|  423|
|  4.0|       4.0|  294|
|  0.0|       0.0| 1355|
|  3.0|       3.0|  870|
+-----+----------+-----+



In [ ]:
print('Accuracy of model: ', round(acc_lg*100,2), '%')

Accuracy of model:  53.48 %


**Nhận xét:**
+ Kết quả đánh giá trên test set cho thấy model có khả năng dự đoán Sentiment thuộc class 0 - Extremely Negative là tốt nhất, tiếp theo là class 3 - Positive. Các kết quả dự đoán trên các class khác là chưa tốt

## Đánh giá trên Unlabeled dataset

In [ ]:
# Check dataset with LogisticRegression model
predictions = lg_model.transform(new_data)
predictions.select('OriginalTweet_clean','probability','prediction').show(10, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------+
|OriginalTweet_clean                                                                                                                                                                                                                                       |probability                                                                                         |prediction|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------

## Lưu và Tải model

In [ ]:
lg_model.save('/content/gdrive/MyDrive/LDS9_K269_ONLINE_NgoThiNga/LDS9_K269_NgoThiNga_Cuoi_ky/Cau2_model')